## Homework 02 - Samuel Zeleke - CS344

#### Exercise 1
Build a spam filter based on Paul Graham’s A Plan for Spam. You’ll find a sketch of his statistical algorithm early in the article (roughly one-fifth of the way through the article).

Include in your solution, as one test case, the probability tables for the words in the following hard-coded SPAM/HAM corpus (and only this corpus) using a minimum count threshold of 1 (rather than the 5 used in the algorithm):

spam_corpus = [["I", "am", "spam", "spam", "I", "am"], ["I", "do", "not", "like", "that", "spamiam"]]

ham_corpus = [["do", "i", "like", "green", "eggs", "and", "ham"], ["i", "do"]]

ANSWER:

In [2]:
import pprint

prettier = pprint.PrettyPrinter()

probTable = dict()

spam_corpus = [["I", "am", "spam", "spam", "I", "am"], ["I", "do", "not", "like", "that", "spamiam"]]
ham_corpus = [["do", "i", "like", "green", "eggs", "and", "ham"], ["i", "do"]]

def spamFilter(spam, ham):
    wordStats = dict()

    # count word in spam
    for document in spam:
        for word in document:
            if word in wordStats.keys():
                wordStats[word]["spam"] += 1
            else:
                wordStats[word] = {"spam": 1, "ham": 0}

    # count word in ham
    for document in ham:
        for word in document:
            if word in wordStats.keys():
                wordStats[word]["ham"] += 1
            else:
                wordStats[word] = {"ham": 1, "spam": 0}

    # calculate the probabilities
    returnContainer = dict()

    for word in wordStats:
        if wordStats[word]["spam"] + wordStats[word]["ham"] > 1:
            statSpam = 1 if wordStats[word]["spam"] / len(spam) > 1 else wordStats[word]["spam"] / len(spam)
            statHam = 1 if 2*wordStats[word]["ham"]/len(ham) > 1 else 2*wordStats[word]["ham"]/len(ham)
            prob = statSpam / (statHam + statSpam)

            returnContainer[word] = (prob if prob > 0.01 else 0.01) if prob < 0.99 else 0.99


    return returnContainer

spamFilter(spam_corpus, ham_corpus)

{'I': 0.99,
 'am': 0.99,
 'do': 0.3333333333333333,
 'i': 0.01,
 'like': 0.3333333333333333,
 'spam': 0.99}


Graham argues that this is a Baysian approach to SPAM. What makes it Bayesian?

ANSWER:
    The approach evaluates an email based on the conditional probabilities of a select number of words, instead of using
    a complete joint probability table for the words used in emails. 
    
#### Exercise 2